# Encoder test
Testing encoder with data from the data_processing module

Instructions:
 - Upload DataProcessing.py, config.yml, process.py, utils.py and encoder.py
 - In DataProcessing.py, change *from data_processing.utils import ** to *from utils import **.

### Imports

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install tensorflow-addons
!{sys.executable} -m pip install -q tensorflow-io
!{sys.executable} -m pip install pyyaml

In [2]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow_io as tfio
import tensorflow as tf
import importlib
import yaml

from google.colab import auth
from pathlib import Path
from tqdm import tqdm 
from PIL import Image

In [4]:
auth.authenticate_user()

In [5]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [6]:
!mkdir DataSet
!gcsfuse --implicit-dirs spade_dataset DataSet

2021/04/30 07:12:02.449791 Using mount point: /content/DataSet
2021/04/30 07:12:02.458278 Opening GCS connection...
2021/04/30 07:12:02.686089 Mounting file system "spade_dataset"...
2021/04/30 07:12:02.703014 File system has been successfully mounted.


### Process data and load dataset

In [7]:
import DataProcessing as dp 
from encoder import Encoder

In [8]:
config = yaml.load(Path("config.yml").read_text(), Loader=yaml.SafeLoader)

In [9]:
set_type = "val"
base_path = "DataSet/cityscape/processed_data"
reader = dp.DataReader(base_path, set_type)

In [10]:
reader.read_data_set()
data_set = reader.get_dataset()
type(data_set)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [32]:
for example in data_set.take(1):
  print(example.keys())
  print(example["label"])
  print(example["subset"])

  img_masked = example["img_masked"]
  img_original = example["img_original"]

  
  # Check images.
  img_masked = Image.fromarray(img_masked.numpy(), 'RGB')
  img_masked.save("img_masked.jpg")

  img_original = Image.fromarray(img_original.numpy(), 'RGB')
  img_original.save("img_original.jpg")

dict_keys(['label', 'subset', 'img_masked', 'img_original'])
tf.Tensor(b'frankfurt_000000_000294', shape=(), dtype=string)
tf.Tensor(b'val', shape=(), dtype=string)


### Encode

In [19]:
encoder = Encoder()

In [30]:
encoder(tf.reshape(img_original, (-1, *img_original.shape)))[0].shape

TensorShape([1, 256])